<a href="https://colab.research.google.com/github/Freyaah/python_for_data_analysis_2nd_chinese_version/blob/master/Summarization_Rouge_Evaluation_24Feb2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libaries

useful links

https://huggingface.co/transformers/v3.0.2/model_doc/bert.html

explanation on rouge differences for extractive vs abstractive summarization

https://stats.stackexchange.com/questions/451419/rouge-scores-for-extractive-vs-abstractive-text-summarization

website about abstractive summarization

https://medium.com/sciforce/towards-automatic-summarization-part-2-abstractive-methods-c424386a65ea

BigBirdPegasus: can accomodate up to 4096 vs 512 

https://huggingface.co/google/bigbird-pegasus-large-pubmed

***Potential models you may want to try can be found:***

***https://paperswithcode.com/sota/text-summarization-on-pubmed-1***

article that we are using to create the reference summary for history of presenting illness

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9285173/

In [7]:
#! pip install bert-extractive-summarizer
#! pip install evaluate
! pip install pysummarization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 KB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysummarization: filename=pysummarization-1.1.9-py3-none-any.whl size=82376 sha256=92f8fde4d4877ec5226dd2f2166cbe624ef32584282ddc5640e19ded29c44347
  Stored in directory: /root/.cache/pip/wheels/b5/28/29/9bad07ea24d08ecc60fc05419196e0e2506dc1e0eb121b0d31
Successfully built pysummarization


In [6]:
import pandas as pd
import re
import matplotlib

# ensure you have the summarizer package installed
# ! pip install bert-extractive-summarizer

from summarizer import Summarizer

from transformers import BertTokenizer, GPT2Tokenizer, EncoderDecoderModel
from transformers import AutoTokenizer, AutoModel, AutoModelWithLMHead

# use t5 in tf
import torch
import torch.nn.functional as F

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from transformers import T5Tokenizer, T5ForConditionalGeneration

import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, LongT5ForConditionalGeneration

from pysummarization.nlpbase.auto_abstractor import AutoAbstractor
from pysummarization.tokenizabledoc.simple_tokenizer import SimpleTokenizer
from pysummarization.abstractabledoc.top_n_rank_abstractor import TopNRankAbstractor
import regex as re

## for bart
import transformers  #(3.0.1)

import gensim
from gensim.summarization import summarize

# to use rouge you need to install it
# ! pip install rouge
from rouge import Rouge 

ModuleNotFoundError: ignored

In [ ]:
# if using GPU, this is to check if the cuda installation is properly set up for pytorch
import torch
print(torch.cuda.is_available())

In [ ]:
import gc
##del model when no memory, can use this instead of restarting jupyter.
gc.collect()
torch.cuda.empty_cache()

# Import Dataset

In [ ]:
hpi_df = pd.read_csv('./datasets/cohort_hpi_df.csv',index_col=0)
hpi_df.shape
# #taking a small subset of 10 examples
# shpi=hpi_df.iloc[0:10,:]

In [ ]:
shpi= pd.read_csv('./datasets/shpi.csv')
shpi.head(2)
# shpi=shpi.iloc[0:10,:]

# BERT Summarizer

In [ ]:
from summarizer import Summarizer

model = Summarizer()

def BertSummarizer(text):
    result = model(text,ratio=0.2)
    
    return result

In [ ]:
shpi['BertSummarizer'] = shpi['hpi_input_text'].apply(BertSummarizer)
shpi['rougeBertSummarizer'] = shpi.apply(lambda x: calculate_rouge(x['BertSummarizer'], x['hpi_reference_summary']), axis=1)

# BERTGPT2 Model using bioclinical tokenizer

paper regarding Bioclinical BERT
https://arxiv.org/abs/1904.03323

In [ ]:
# tokenizer for bioclinical bert
bioclinberttokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2gpt2-cnn_dailymail-fp16")
model.to("cuda")

#bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
bert_tokenizer= AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# CLS token will work as BOS token
bert_tokenizer.bos_token = bert_tokenizer.cls_token

# SEP token will work as EOS token
bert_tokenizer.eos_token = bert_tokenizer.sep_token


# make sure GPT2 appends EOS in begin and end
def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
    outputs = [self.bos_token_id] + token_ids_0 + [self.eos_token_id]
    return outputs


GPT2Tokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# set pad_token_id to unk_token_id -> be careful here as unk_token_id == eos_token_id == bos_token_id
gpt2_tokenizer.pad_token = gpt2_tokenizer.unk_token


# set decoding params
model.config.decoder_start_token_id = gpt2_tokenizer.bos_token_id
model.config.eos_token_id = gpt2_tokenizer.eos_token_id
model.config.max_length = 142
model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.early_stopping = True
model.length_penalty = 2.0
model.num_beams = 4

#test_dataset = nlp.load_dataset("cnn_dailymail", "3.0.0", split="test")

batch_size = 64

In [ ]:
def generate_BERT2GPT2_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = bert_tokenizer(batch, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = gpt2_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    #batch["pred"] = output_str

    return output_str

In [ ]:
shpi['BertGPT2'] = shpi['hpi_input_text'].apply(generate_BERT2GPT2_summary)

# T5

## T5 Seq2Seq

In [ ]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

In [ ]:
def seq2seq(text):

    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    summary= tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return summary

In [ ]:
shpi['t5seq2eq'] = shpi['hpi_input_text'].apply(seq2seq)

## T5 Base

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

t5_model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)
t5_tokenizer = T5Tokenizer.from_pretrained("t5-base")
num_beams=2
min_length=15
max_length=200

model='t5'
if model == 't5':
    _num_beams = 1
    _no_repeat_ngram_size = 1
    _length_penalty = 2
    _min_length = 30
    _max_length = 200
    _early_stopping = True

def t5(input_text):
    input_text = str(input_text).replace('\n', '')
    input_text = ' '.join(input_text.split())
    input_tokenized = t5_tokenizer.encode(input_text, return_tensors="pt").to(device)
    summary_task = torch.tensor([[21603, 10]]).to(device)
    input_tokenized = torch.cat([summary_task, input_tokenized], dim=-1).to(device)
    summary_ids = t5_model.generate(input_tokenized,
                                        num_beams=_num_beams,
                                        no_repeat_ngram_size=_no_repeat_ngram_size,
                                        length_penalty=_length_penalty,
                                        min_length=_min_length,
                                        max_length=_max_length,
                                        early_stopping=_early_stopping)
    output = [t5_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    return output

In [ ]:
shpi['t5'] = shpi['hpi_input_text'].apply(t5)

## Long T5

In [ ]:
model = (
    LongT5ForConditionalGeneration.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")
    .to("cuda")
    .half()
)
tokenizer = AutoTokenizer.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")

In [ ]:
def generate_answers_longT5(batch):
    inputs_dict = tokenizer(
        batch, max_length=500, padding="max_length", truncation=True, return_tensors="pt"
    )
    input_ids = inputs_dict.input_ids.to("cuda")
    attention_mask = inputs_dict.attention_mask.to("cuda")
    output_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=200, num_beams=4)
    summary = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    return summary

In [ ]:
shpi['longt5'] = shpi['hpi_input_text'].apply(generate_answers_longT5)

# PySummarizer

https://pypi.org/project/pysummarization/

In [ ]:
auto_abstractor = AutoAbstractor()
auto_abstractor.tokenizable_doc = SimpleTokenizer()
auto_abstractor.delimiter_list = [".", "\n"]
abstractable_doc = TopNRankAbstractor()

In [ ]:
shpi['transcription'] = shpi['hpi_input_text'].astype(str)

In [ ]:
def pysummarizer(text):
#    print(type(text))
    summary = auto_abstractor.summarize(text, abstractable_doc)
    best_sentences=[]
    #summary_clean = ''.join([str(sentence).capitalize() for sentence in summary['summarize_result'] for summary['summarize_result'] in auto_abstractor.summarize(text, abstractable_doc)])
    for sentence in summary['summarize_result']:
        best_sentences.append(re.sub(r'\s+', ' ', sentence).strip())    
    clean_summary=''.join(sentence for sentence in best_sentences)
    return clean_summary


In [ ]:
shpi['pysummarizer'] = shpi['transcription'].apply(pysummarizer)
shpi = shpi.drop('transcription', axis=1)

# Bart

In [ ]:
'''
Summarizes corpus with Bart.
:parameter    
   :param corpus: list - dtf["text"]    
   :param max_len: length of the summary
:return    
    list of summaries
'''
def bart(corpus, max_len):    
    nlp = transformers.pipeline("summarization")    
    lst_summaries = [nlp(txt,               
                         max_length=max_len
                         )[0]["summary_text"].replace(" .", ".")                    
                     for txt in corpus]    
    return lst_summaries

## Apply the function to corpus
predicted = bart(corpus=shpi['hpi_input_text'], max_len=100)

# Gensim

https://radimrehurek.com/gensim_3.8.3/summarization/summariser.html

In [ ]:
shpi['gensim'] = ''
for (idx, row) in shpi.iterrows():
#     print(idx)
    try:
        input_text = shpi['hpi_input_text'][idx]
        shpi['gensim'][idx] = summarize(str(input_text))
    except:
        shpi['gensim'][idx] = row.loc['gensim']
        print(row.loc['gensim'],row)

# Generate Rouge Score

In [ ]:
rouge = Rouge()

def calculate_rouge(predicted, reference):

    scores = rouge.get_scores(predicted, reference)
    return scores

In [ ]:
modelsrouge=['BertSummarizer','BertGPT2','t5seq2eq','t5','gensim','pysummarizer']

for model in modelsrouge:
    shpi['rouge'+model] = ''
    for (idx, row) in shpi.iterrows():
        try:
            shpi['rouge'+model][idx] = calculate_rouge(shpi[model][idx], shpi['hpi_reference_summary'][idx])
        except:
            shpi['rouge'+model][idx] = ''   

In [ ]:
for model in modelsrouge:
    shpi['r1_recall_'+model] = ''
    shpi['r1_precision_'+model] = ''
    shpi['r1_f_'+model] = ''
    shpi['r2_recall_'+model] = ''
    shpi['r2_precision_'+model] = ''
    shpi['r2_f_'+model] = ''
    shpi['rl_recall_'+model] = ''
    shpi['rl_precision_'+model] = ''
    shpi['rl_f_'+model] = ''

    for (idx, row) in shpi.iterrows():
        try: 
            rs = shpi['rouge'+model][idx][0]
            shpi['r1_recall_'+model][idx] = rs['rouge-1']['r']
            shpi['r1_precision_'+model][idx] = rs['rouge-1']['p']
            shpi['r1_f_'+model][idx] = rs['rouge-1']['f']

            shpi['r2_recall_'+model][idx] = rs['rouge-2']['r']
            shpi['r2_precision_'+model][idx] = rs['rouge-2']['p']
            shpi['r2_f_'+model][idx] = rs['rouge-2']['f']

            shpi['rl_recall_'+model][idx] = rs['rouge-l']['r']
            shpi['rl_precision_'+model][idx] = rs['rouge-l']['p']
            shpi['rl_f_'+model][idx] = rs['rouge-l']['f']
        except:
            shpi['r1_recall_'+model][idx] = 0
            shpi['r1_precision_'+model][idx] = 0
            shpi['r1_f_'+model][idx] = 0

            shpi['r2_recall_'+model][idx] = 0
            shpi['r2_precision_'+model][idx] = 0
            shpi['r2_f_'+model][idx] = 0

            shpi['rl_recall_'+model][idx] = 0
            shpi['rl_precision_'+model][idx] = 0
            shpi['rl_f_'+model][idx] = 0

In [ ]:
shpi

In [ ]:
pd.pandas.set_option('display.max_columns', None)
shpi.head()